In [1]:
## Se importan los paquetes necesarios para trabajar
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import os

In [2]:
## Se escrpean los datos del sitio web 'https://eloratings.net/'. Se obtienen los registros de las posiciones en el ranking
## Elo de los paises de la CONMEBOL. Los campos que se obtienen son: año, equiponac (equipo nacional), rank (puesto en el
## ranking Elo).

chrome_driver = '*:/***/****/****/chromedriver' ## Aquí se debe incluir la dirección al webdriver para utilizar Selenium.
                                                 ## Para mayor información: https://www.seleniumhq.org/projects/webdriver/
                                                 ## Yo utilizo Chrome por lo que el webdriver es 'chromedriver'.
        
driver = webdriver.Chrome(chrome_driver)
driver.implicitly_wait(10) 
equipos = ['Bolivia','Colombia','Argentina','Ecuador','Uruguay','Peru','Chile','Venezuela','Paraguay','Brazil']

count = 0
fecha = []
año = []
pais1 = []
pais2 = []
rank1 = []
rank2 = []
equiponac = []

for equipo in equipos:
    url = 'https://eloratings.net/'
    driver.get(url)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, equipo))).click()
    tabla = driver.find_element_by_id(str('maintable_{}'.format(equipos[count])))
    r = tabla.get_attribute('innerHTML')
    soup = BeautifulSoup(r,'html.parser')
    dato_fechas = soup.find_all(class_='slick-cell l0 r0 match-cell ')
    dato_paises = soup.find_all(class_='slick-cell l1 r1 match-cell ')
    dato_rank = soup.find_all(class_='slick-cell l7 r7 score-cell ')
    sleep(2)

    for i in range(len(dato_fechas)):
        fecha_año = re.search(r'([A-Za-z]+\s+\d+).+(\d\d\d\d)',str(dato_fechas[i]))
        fecha.append(fecha_año.group(1))
        año.append(fecha_año.group(2))

    for i in range(len(dato_paises)):
        paises = re.search(r'>([A-Za-zç ]+)<.+>([A-Za-zç ]+)<',str(dato_paises[i]))
        pais1.append(paises.group(1))
        pais2.append(paises.group(2))    

    for i in range(len(dato_rank)):
        rankings = re.search(r'>(\d+)<.+>(\d+)<',str(dato_rank[i]))
        rank1.append(rankings.group(1))
        rank2.append(rankings.group(2))

    for i in range(len(dato_rank)):
        equiponac.append(equipos[count])
        
    count += 1

driver.close()

elo = pd.DataFrame({'año':año,'equiponac':equiponac,'pais1':pais1,'pais2':pais2,'rank1':rank1,'rank2':rank2})
elo['rank'] = np.where((elo['equiponac']=='Bolivia')&(elo['pais1']=='Bolivia')|
                       (elo['equiponac']=='Colombia')&(elo['pais1']=='Colombia')|
                       (elo['equiponac']=='Argentina')&(elo['pais1']=='Argentina')|
                       (elo['equiponac']=='Ecuador')&(elo['pais1']=='Ecuador')|
                       (elo['equiponac']=='Uruguay')&(elo['pais1']=='Uruguay')|
                       (elo['equiponac']=='Peru')&(elo['pais1']=='Peru')|
                       (elo['equiponac']=='Chile')&(elo['pais1']=='Chile')|
                       (elo['equiponac']=='Venezuela')&(elo['pais1']=='Venezuela')|
                       (elo['equiponac']=='Paraguay')&(elo['pais1']=='Paraguay')|
                       (elo['equiponac']=='Brazil')&(elo['pais1']=='Brazil')
                                                      ,elo['rank1'],elo['rank2'])
elo = elo.drop(['pais1','pais2','rank1','rank2'],axis=1)
elo['rank'] = elo['rank'].astype(int)
elo_agg = elo.groupby(['equiponac','año']).mean().reset_index()
elo_agg['rank'] = round(elo_agg['rank'],2)

In [3]:
path = os.getcwd()

In [4]:
## Se graba el archivo equiposnac.csv con el dataset.
elo_agg.to_csv(path+'\\elo_agg.csv')